Стохастический градиентый спуск сходится быстрее, чем полный, хотя и менее стабильно. В этом задании вам предлагается реализовать стохастический градиентный спуск и сравнить его с точным вычислением весов линейной модели по скорости работы и значению метрики качества.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings

np.random.seed(0)

warnings.filterwarnings('ignore')
%matplotlib inline

### Задание 0

Реализуйте класс ```LinearRegressionSGD``` c обучением и и применением линейной регрессии, построенной с помощью стохастического градиентного спуска, с заданным интерфейсом.

In [ ]:
from sklearn.base import BaseEstimator

class LinearRegressionSGD(BaseEstimator):
    def __init__(self, epsilon=1e-4, max_steps=100, w0=None, alpha=1e-4):
        """
        epsilon: разница для нормы изменения весов 
        max_steps: максимальное количество шагов в градиентном спуске
        w0: np.array (d,) - начальные веса
        alpha: шаг обучения
        """
        self.epsilon = epsilon
        self.max_steps = max_steps
        self.w0 = w0
        self.alpha = alpha
        self.w = None
        self.w_history = []
        self.grad_history = []
    
    def fit(self, X, y):
        """
        X: np.array (l, d)
        y: np.array (l)
        ---
        output: self
        """
        ## На каждом шаге градиентного спуска веса необходимо добавлять в w_history
        
        l, d = X.shape
        
        if self.w0 is None:
            self.w0 = np.zeros(d)
            
        self.w = self.w0
        
        for i in range(self.max_steps):
            self.w_history.append(self.w)
            
            w_new = self.w - self.alpha * self.calc_gradient(X, y)
            
            if (np.linalg.norm(self.w - w_new) < self.epsilon):
                break
            
            self.w = w_new
        
        return self
    
    def predict(self, X):
        """
        X: np.array (l, d)
        ---
        output: np.array (l)
        """
        
        if self.w is None:
            raise Exception('Not trained yet')
            
        l, d = X.shape
        y_pred = []
        
        for i in range(l):
            y_pred.append(np.dot(X[i], self.w))
            
        #y_pred = np.dot(X, self.w)

        return np.array(y_pred)
    
    def calc_gradient(self, X, y):
        """
        X: np.array (l, d)
        y: np.array (l)
        ---
        output: np.array (d)
        """

        l, d = X.shape
        gradient = []
        randoom_index = np.random.randint(0, l)
        
        for j in range(d):
            dQ = (2/l) * X[randoom_index, j] * (np.dot(X[randoom_index], self.w) - y[randoom_index])
            gradient.append(dQ)
            
        self.grad_history.append(gradient)

        return np.array(gradient)

Проверять работу Вы будете на имеющемся в sklearn наборе данных boston: в нём нужно по информации о доме предсказать его стоимость.

In [ ]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

data = load_boston()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

X_train, X_test, y_train, y_test = train_test_split(np.array(X), y, test_size=0.3, random_state=10)

### Задание 1

Метрикой качества в нашей задаче будет MAPE - Mean Absolute Percentage Error. Реализуйте её с заданным интефейсом и вычислите 
```MAPE(y_test, y_0)```, где ```y_0 = (mean(y_test), mean(y_test), ...)```

In [ ]:
def MAPE(y_true, y_pred):
    """
        y_true: np.array (l)
        y_pred: np.array (l)
        ---
        output: float [0, +inf)
    """
    l = y_true.shape[0]
#     mape = 0
    
#     for i in range(l):
#         mape += abs((y_true[i] - y_pred[i]) / y_true[i])
#     mape /= l
    mape = (1 / l) * np.linalg.norm(abs((y_true - y_pred)) / abs(y_true), ord = 1)
    
    return mape 

In [ ]:
y_0 = np.mean(y_test)

MAPE(y_test, y_0)

0.37415882976840964

### Задание 2 

Обучите ```LinearRegressionSGD``` с базовыми параметрами на тренировочном наборе данных (```X_train```, ```y_train```), сделайте предсказание на тестовых данных ```X_test```, записав результат в переменную ```y_pred_sgd```  и вычислите ошибку MAPE.

In [ ]:
sgd = LinearRegressionSGD(alpha = 1e-5)

sgd.fit(X_train, y_train)

y_pred_sgd = sgd.predict(X_test)

MAPE(y_test, y_pred_sgd)



0.4801742419994511

### Задание 3

Вычислите веса по точной формуле, используя ```X_train``` и ```y_train```; предскажите с их помощью целевую переменную на ```X_test```, записав результат в переменную ```y_pred_lr``` и вычислите ошибку MAPE.

In [ ]:
w = np.dot(np.dot(np.linalg.inv(np.dot(np.transpose(X_train), X_train)), np.transpose(X_train)), y_train)

y_pred_lr = np.dot(X_test, w)

MAPE(y_test, y_pred_lr)

0.18953134816375927

## Задание 4

До этого вы релизовывали модели, в которых не было штрафа за величину весов ```w```. Это может привести к неустойчивости модели и переобучению. Чтобы избежать этих эффектов, предлагается добавить к оптимизируемому функционалу L2-норму весов; таким образом, будем решать задачу гребневой регрессии, Ridge:

$$ \frac{1}{l}(Xw-y)^T(Xw-y) +\gamma||w||_2 \rightarrow \min_{w}. $$


Реализуйте обучение такой модели в матричном виде с помощью стохастического градиентного спуска. Класс должен совпадать по набору реализованных функций с ```LinearRegressionVectorized```, разница будет лишь в параметре $\gamma$, отвечающем за степень регуляризации. 

In [ ]:
from sklearn.base import BaseEstimator
class RidgeSGD(BaseEstimator):
    def __init__(self, epsilon=1e-4, max_steps=1000, w0=None, alpha=1e-2, gamma=0):
        """
        epsilon: разница для нормы изменения весов 
        max_steps: максимальное количество шагов в градиентном спуске
        w0: np.array (d,) - начальные веса
        alpha: шаг обучения
        gamma: коэффициент регуляризации
        """
        self.epsilon = epsilon
        self.max_steps = max_steps
        self.w0 = w0
        self.alpha = alpha
        self.gamma = gamma
        self.w = None
        self.w_history = []
    
    def fit(self, X, y):
        """
        X: np.array (l, d)
        y: np.array (l)
        ---
        output: self
        """
        
        l, d = X.shape
        
        if self.w0 is None:
            self.w0 = np.zeros(d)
            
        self.w = self.w0
        
        for step in range(self.max_steps):
            self.w_history.append(self.w)
            
            new_w = self.w - self.alpha * self.calc_gradient(X, y)
            
            if (np.linalg.norm(self.w - new_w) < self.epsilon):
                break
            
            self.w = new_w
        
        return self
    
    def predict(self, X):
        """
        X: np.array (l, d)
        ---
        output: np.array (l)
        """
        
        if self.w is None:
            raise Exception('Not trained yet')
        
        return np.dot(X, self.w)

    
    def calc_gradient(self, X, y):
        """
        X: np.array (l, d)
        y: np.array (l)
        ---
        output: np.array (d)
        """
        
        l, d = X.shape
        rand_i = np.random.randint(0, l)
        
        return (2/l) * np.transpose(X[rand_i]) * (np.dot(X[rand_i], self.w) - y[rand_i])\
    + self.gamma * (np.dot(np.transpose(self.w), self.w)) * self.w

Обучите модель с базовыми параметрами на тренировочных данных и сделайте прогноз y_pred_ridge. Выведите значение MAPE(y_test, y_pred_ridge).

In [ ]:
rsgd = RidgeSGD(alpha = 1e-4)

rsgd.fit(X_train, y_train)

y_pred_ridge = rsgd.predict(X_test)
MAPE(y_test, y_pred_ridge)

0.3347412926099105